# 1. multiprocessing example

Parallel processing is a mode of operation where the task is executed simultaneously in multiple processors in the same computer. It is meant to reduce the overall processing time.

However, there is usually a bit of overhead when communicating between processes which can actually increase the overall time taken for small tasks instead of decreasing it.

In python, the  module is used to run independent parallel processes by using subprocesses (instead of threads). It allows you to leverage multiple processors on a machine (both Windows and Unix), which means, the processes can be run in completely separate memory locations.




# 2. How many maximum parallel processes can you run?

In [ ]:
import multiprocessing as mp

print("Number of processors: ", mp.cpu_count())

# 3. What is Synchronous and Asynchronous execution?
In parallel processing, there are two types of execution: Synchronous and Asynchronous.

A synchronous execution is one the processes are completed in the same order in which it was started. This is achieved by locking the main program until the respective processes are finished.

Asynchronous, on the other hand, doesn’t involve locking. As a result, the order of results can get mixed up but usually gets done quicker.

There are 2 main objects in multiprocessing to implement parallel execution of a function: The Pool Class and the Process Class.

Pool Class
Synchronous execution
Pool.map() and Pool.starmap()
Pool.apply()
Asynchronous execution
Pool.map_async() and Pool.starmap_async()
Pool.apply_async())
Process Class

# 4. Problem Statement: Count how many numbers exist between a given range in each row
The first problem is: Given a 2D matrix (or list of lists), count how many numbers are present between a given range in each row. We will work on the list prepared below.

In [8]:
import numpy as np
from time import time

# Prepare data
np.random.RandomState(100)
arr = np.random.randint(0, 10, size=[200000, 5])
data = arr.tolist()
data[:5]

[[1, 2, 6, 2, 3],
 [1, 3, 6, 5, 2],
 [6, 6, 6, 7, 0],
 [9, 0, 2, 3, 6],
 [0, 6, 0, 8, 1]]

## Solution without parallelization

In [11]:
%%time
# Solution Without Paralleization

def howmany_within_range(row, minimum, maximum):
    """Returns how many numbers lie within `maximum` and `minimum` in a given `row`"""
    count = 0
    for n in row:
        if minimum <= n <= maximum:
            count = count + 1
    return count

results = []
for row in data:
    results.append(howmany_within_range(row, minimum=4, maximum=8))

print(results[:10])

[1, 2, 4, 1, 2, 2, 2, 3, 3, 3]
CPU times: user 170 ms, sys: 44.9 ms, total: 215 ms
Wall time: 213 ms


# 5. How to parallelize any function?
The general way to parallelize any operation is to take a particular function that should be run multiple times and make it run parallelly in different processors.

To do this, you initialize a Pool with n number of processors and pass the function you want to parallelize to one of Pools parallization methods.

multiprocessing.Pool() provides the apply(), map() and starmap() methods to make any function run in parallel.

Nice! So what’s the difference between apply() and map()?

Both apply and map take the function to be parallelized as the main argument. But the difference is, apply() takes an args argument that accepts the parameters passed to the ‘function-to-be-parallelized’ as an argument, whereas, map can take only one iterable as an argument.

So, map() is really more suitable for simpler iterable operations but does the job faster.

We will get to starmap() once we see how to parallelize howmany_within_range() function with apply() and map().

## 5.1. Parallelizing using Pool.apply()

In [12]:
%%time
# Parallelizing using Pool.apply()

# Step 1: Init multiprocessing.Pool()
pool = mp.Pool(mp.cpu_count())

# Step 2: `pool.apply` the `howmany_within_range()`
results = [pool.apply(howmany_within_range, args=(row, 4, 8)) for row in data]

# Step 3: Don't forget to close
pool.close()    

print(results[:10])



[1, 2, 4, 1, 2, 2, 2, 3, 3, 3]
CPU times: user 29.2 s, sys: 12.6 s, total: 41.8 s
Wall time: 53.7 s


## 5.2. Parallelizing using Pool.map()
Pool.map() accepts only one iterable as argument. 

In [ ]:
%%time
# Parallelizing using Pool.map()
import multiprocessing as mp

# Redefine, with only 1 mandatory argument.
def howmany_within_range_rowonly(row, minimum=4, maximum=8):
    count = 0
    for n in row:
        if minimum <= n <= maximum:
            count = count + 1
    return count

pool = mp.Pool(mp.cpu_count())

results = pool.map(howmany_within_range_rowonly, [row for row in data])
#results = [pool.apply(howmany_within_range_rowonly, args=(row, )) for row in data]

pool.close()

print(results[:10])

## 5.3. Parallelizing using Pool.starmap()

In previous example, we have to redefine howmany_within_range function to make couple of parameters to take default values. Using starmap(), you can avoid doing this. How you ask?

Like Pool.map(), Pool.starmap() also accepts only one iterable as argument, but in starmap(), each element in that iterable is also a iterable. You can to provide the arguments to the ‘function-to-be-parallelized’ in the same order in this inner iterable element, will in turn be unpacked during execution.

So effectively, Pool.starmap() is like a version of Pool.map() that accepts arguments.


In [ ]:
%%time
# Parallelizing with Pool.starmap()
import multiprocessing as mp


pool = mp.Pool(mp.cpu_count())

results = pool.starmap(howmany_within_range, [(row, 4, 8) for row in data])

pool.close()

print(results[:10])

let's put them together:
  pool.apply(howmany_within_range, args=(row, 4, 8)) for row in data
  pool.map(howmany_within_range_rowonly, [row for row in data])
  pool.starmap(howmany_within_range, [(row, 4, 8) for row in data])

# 6. Asynchronous Parallel Processing
The asynchronous equivalents apply_async(), map_async() and starmap_async() lets you do execute the processes in parallel asynchronously, that is the next process can start as soon as previous one gets over without regard for the starting order. As a result, there is no guarantee that the result will be in the same order as the input.

## 6.1 Parallelizing with Pool.apply_async()



In [4]:
%%time
# Parallel processing with Pool.apply_async()

import multiprocessing as mp
pool = mp.Pool(mp.cpu_count())

results = []

# Step 1: Redefine, to accept `i`, the iteration number
def howmany_within_range2(i, row, minimum, maximum):
    """Returns how many numbers lie within `maximum` and `minimum` in a given `row`"""
    count = 0
    for n in row:
        if minimum <= n <= maximum:
            count = count + 1
    return (i, count)

#> [3, 1, 4, 4, 4, 2, 1, 1, 3, 3]

CPU times: user 9.08 ms, sys: 13 ms, total: 22.1 ms
Wall time: 49.3 ms


In [5]:
# Step 2: Define callback function to collect the output in `results`
def collect_result(result):
    global results
    results.append(result)


# Step 3: Use loop to parallelize
for i, row in enumerate(data):
    pool.apply_async(howmany_within_range2, args=(i, row, 4, 8), callback=collect_result)



In [6]:
# Step 4: Close Pool and let all the processes complete    
pool.close()
pool.join()  # postpones the execution of next line of code until all processes in the queue are done.

# Step 5: Sort results [OPTIONAL]
results.sort(key=lambda x: x[0])
results_final = [r for i, r in results]

print(results_final[:10])

[2, 3, 4, 3, 2, 2, 3, 2, 2, 2]


In [7]:
# Parallel processing with Pool.apply_async() without callback function

import multiprocessing as mp
pool = mp.Pool(mp.cpu_count())

results = []

# call apply_async() without callback
result_objects = [pool.apply_async(howmany_within_range2, args=(i, row, 4, 8)) for i, row in enumerate(data)]

# result_objects is a list of pool.ApplyResult objects
results = [r.get()[1] for r in result_objects]

pool.close()
pool.join()
print(results[:10])

[2, 3, 4, 3, 2, 2, 3, 2, 2, 2]



## 6.2 Parallelizing with Pool.starmap_async()
You saw how apply_async() works. Can you imagine and write up an equivalent version for starmap_async and map_async? The implementation is below anyways.

In [ ]:
# Parallelizing with Pool.starmap_async()

import multiprocessing as mp
pool = mp.Pool(mp.cpu_count())

results = []

results = pool.starmap_async(howmany_within_range2, [(i, row, 4, 8) for i, row in enumerate(data)]).get()

# With map, use `howmany_within_range_rowonly` instead
# results = pool.map_async(howmany_within_range_rowonly, [row for row in data]).get()

pool.close()
print(results[:10])
#> [3, 1, 4, 4, 4, 2, 1, 1, 3, 3]